In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

df= pd.read_csv('C://Users//User//Desktop//MSc Westminster//Dissertation//DataSets//Heart_Attack_Prediction.csv')

In [2]:
columns=df.columns

for column in columns:
    if df[column].dtype=="int32":
        df[column]=df[column].astype("int16")
    elif df[column].dtype=="float64":
        df[column]=df[column].astype("float16")
    elif df[column].dtype=="object":
        df[column]=df[column].astype("category")

In [3]:
df['Sex']= df['Sex'].map({'Female': 0, 'Male': 1})
df['Sex']= pd.to_numeric(df['Sex'])

df['Diet']= df['Diet'].map({'Healthy': 0, 'Average': 1, 'Unhealthy':2})
df['Diet']= pd.to_numeric(df['Diet'])

df[['HBP', 'LBP']]= df['Blood Pressure'].str.split('/', expand= True)
df['HBP']= pd.to_numeric(df['HBP'])
df['LBP']= pd.to_numeric(df['LBP'])

df['Diabetes'] = df['Diabetes'].map({0: 1, 1: 0})

df['Exercise Hours Per Week']= round(df['Exercise Hours Per Week'], 0)

df['Sedentary Hours Per Day']= round(df['Sedentary Hours Per Day'], 0)

df['Income']= round(df['Income'], 0)

df['BMI']= round(df['BMI'], 0)

df = df.drop(columns=['Patient ID', 'Blood Pressure', 'Country', 'Continent', 'Hemisphere'])

In [4]:
X= df.drop(['Heart Attack Risk'], axis= 1)
y= df['Heart Attack Risk']

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size= 0.2, random_state= 15, stratify= y)

from sklearn.preprocessing import RobustScaler
scaler= RobustScaler()
scaler.fit(X_train)

X_train= scaler.transform(X_train)
X_test= scaler.transform(X_test)

In [6]:
from imblearn.under_sampling import ClusterCentroids
from imblearn.over_sampling import SMOTE
from sklearn.cluster import KMeans

sm= ClusterCentroids(random_state= 15, estimator= KMeans(n_init= 10))
tl= SMOTE(random_state= 15)

X_sm, y_sm= sm.fit_resample(X_train, y_train)
X_sm_tl, y_sm_tl= tl.fit_resample(X_sm, y_sm)

In [7]:
def backward_elimination(data, target, significance_level= 0.05):
    features= data.columns.tolist()
    while len(features) > 0:
        model= sm.Logit(target, data[features])
        result= model.fit(disp=0)
        p_values= result.pvalues
        max_p_value= p_values.max()
        if max_p_value > significance_level:
            excluded_feature= p_values.idxmax()
            print(f"Excluding {excluded_feature} with p-value {max_p_value}")
            features.remove(excluded_feature)
        else:
            break
    return data[features]

In [8]:
import statsmodels.api as sm

X_sm_tl_df= pd.DataFrame(X_sm_tl, columns= X.columns)
X_test_df= pd.DataFrame(X_test, columns= X.columns)

selected_features= backward_elimination(X_sm_tl_df, y_sm_tl)
X_sm_tl_selected= X_sm_tl_df[selected_features.columns]
X_sm_tl_test_selected= X_test_df[selected_features.columns]

Excluding BMI with p-value 0.9868371055111534
Excluding Income with p-value 0.9590482832562202
Excluding Smoking with p-value 0.9235855346391468
Excluding Alcohol Consumption with p-value 0.7234468308245944
Excluding Triglycerides with p-value 0.704656852032463
Excluding Previous Heart Problems with p-value 0.5418184163262065
Excluding Family History with p-value 0.43437778009924144
Excluding Diet with p-value 0.38630899433739097
Excluding Exercise Hours Per Week with p-value 0.3464895853436627
Excluding LBP with p-value 0.33149741994692283
Excluding Heart Rate with p-value 0.272251219965844
Excluding Medication Use with p-value 0.23889956295610126
Excluding Age with p-value 0.17911499188348756
Excluding HBP with p-value 0.17886040613407517
Excluding Physical Activity Days Per Week with p-value 0.13957853772414788
Excluding Cholesterol with p-value 0.12511033298152902
Excluding Sedentary Hours Per Day with p-value 0.12671712613258895
Excluding Stress Level with p-value 0.12094871988953

--- LogisticRegression ---

In [9]:
from sklearn.metrics import confusion_matrix, classification_report, precision_score, roc_auc_score, accuracy_score
from sklearn.linear_model import LogisticRegression
lr= LogisticRegression(random_state= 15)
lr.fit(X_sm_tl_selected, y_sm_tl)
y_pred_lr= lr.predict(X_sm_tl_test_selected)
y_pred_prob_lr= lr.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_lr, labels= lr.classes_)
print(classification_report(y_test, y_pred_lr, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_lr, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_lr))
print('AUC:', roc_auc_score(y_test, y_pred_prob_lr))

              precision    recall  f1-score   support

           0       0.66      0.48      0.56      1125
           1       0.37      0.54      0.44       628

    accuracy                           0.51      1753
   macro avg       0.51      0.51      0.50      1753
weighted avg       0.55      0.51      0.52      1753

Precision: 0.37093275488069416
Accuracy: 0.5059897318881916
AUC: 0.5164748761500354


--- DecisionTreeClassifier ---

In [10]:
from sklearn.tree import DecisionTreeClassifier
dt= DecisionTreeClassifier(random_state=15)
dt.fit(X_sm_tl_selected, y_sm_tl)
y_pred_dt= dt.predict(X_sm_tl_test_selected)
y_pred_prob_dt= dt.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_dt, labels= dt.classes_)
print(classification_report(y_test, y_pred_dt, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_dt, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_dt))
print('AUC:', roc_auc_score(y_test, y_pred_prob_dt))

              precision    recall  f1-score   support

           0       0.65      0.13      0.22      1125
           1       0.36      0.87      0.51       628

    accuracy                           0.40      1753
   macro avg       0.50      0.50      0.36      1753
weighted avg       0.55      0.40      0.32      1753

Precision: 0.35952848722986247
Accuracy: 0.3970336565887051
AUC: 0.504232837933475


--- Tuned - DecisionTreeClassifier ---

In [11]:
from sklearn.model_selection import GridSearchCV
param_grid= {
             'max_depth': [None, 10, 20, 30, 40, 50],
             'min_samples_split': [2, 10, 20],
             'min_samples_leaf': [1, 5, 10],
             'max_features': [None, 'sqrt', 'log2'],
             'criterion': ['gini', 'entropy']
            }

gs_dt= GridSearchCV(estimator= dt, param_grid= param_grid, cv= 5, scoring= 'precision')
gs_dt.fit(X_sm_tl, y_sm_tl)

print("Best Parameters:", gs_dt.best_params_)
print("Best Precision Score:", gs_dt.best_score_)

Best Parameters: {'criterion': 'entropy', 'max_depth': 10, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 2}
Best Precision Score: 0.602544479474124


In [12]:
tuned_dt= gs_dt.best_estimator_
tuned_dt.fit(X_sm_tl_selected, y_sm_tl)
y_pred_tuned_dt= tuned_dt.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_dt= tuned_dt.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_dt, labels= tuned_dt.classes_)
print(classification_report(y_test, y_pred_tuned_dt, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_dt, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_dt))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_dt))

              precision    recall  f1-score   support

           0       0.66      0.15      0.24      1125
           1       0.36      0.86      0.51       628

    accuracy                           0.40      1753
   macro avg       0.51      0.50      0.37      1753
weighted avg       0.55      0.40      0.34      1753

Precision: 0.3606666666666667
Accuracy: 0.4033086138049059
AUC: 0.5091365888181175


--- KNeighborsClassifier ---

In [13]:
from sklearn.neighbors import KNeighborsClassifier
knn= KNeighborsClassifier()
knn.fit(X_sm_tl_selected, y_sm_tl)
y_pred_knn= knn.predict(X_sm_tl_test_selected)
y_pred_prob_knn= knn.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_knn, labels= knn.classes_)
print(classification_report(y_test, y_pred_knn, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_knn, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_knn))
print('AUC:', roc_auc_score(y_test, y_pred_prob_knn))

              precision    recall  f1-score   support

           0       0.65      0.86      0.74      1125
           1       0.38      0.16      0.22       628

    accuracy                           0.61      1753
   macro avg       0.51      0.51      0.48      1753
weighted avg       0.55      0.61      0.55      1753

Precision: 0.38372093023255816
Accuracy: 0.607529948659441
AUC: 0.5081549893842887


--- Tuned - KNeighborsClassifier ---

In [14]:
param_grid= {
             'n_neighbors': np.arange(1,40),
             'weights': ['uniform', 'distance'],
             'metric': ['euclidean', 'manhattan', 'minkowski']
            }

gs_knn= GridSearchCV(estimator= knn, param_grid= param_grid, cv=5, scoring= 'precision')
gs_knn.fit(X_sm_tl, y_sm_tl)
print("Best Parameters:", gs_knn.best_params_)
print("Best Precision Score:", gs_knn.best_score_)

Best Parameters: {'metric': 'manhattan', 'n_neighbors': 38, 'weights': 'uniform'}
Best Precision Score: 0.5348459912753073


In [15]:
tuned_knn= gs_knn.best_estimator_
tuned_knn.fit(X_sm_tl_selected, y_sm_tl)

y_pred_tuned_knn= tuned_knn.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_knn= tuned_knn.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_knn, labels= tuned_knn.classes_)
print(classification_report(y_test, y_pred_tuned_knn, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_knn, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_knn))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_knn))

              precision    recall  f1-score   support

           0       0.65      0.81      0.72      1125
           1       0.39      0.22      0.28       628

    accuracy                           0.60      1753
   macro avg       0.52      0.51      0.50      1753
weighted avg       0.56      0.60      0.56      1753

Precision: 0.39154929577464787
Accuracy: 0.5978322875071307
AUC: 0.49383156404812456


--- GaussianNB ---

In [16]:
from sklearn.naive_bayes import GaussianNB
nb= GaussianNB()
nb.fit(X_sm_tl_selected, y_sm_tl)
y_pred_nb= nb.predict(X_sm_tl_test_selected)
y_pred_prob_nb= nb.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_nb, labels= nb.classes_)
print(classification_report(y_test, y_pred_nb, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_nb, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_nb))
print('AUC:', roc_auc_score(y_test, y_pred_prob_nb))

              precision    recall  f1-score   support

           0       0.66      0.48      0.56      1125
           1       0.37      0.54      0.44       628

    accuracy                           0.51      1753
   macro avg       0.51      0.51      0.50      1753
weighted avg       0.55      0.51      0.52      1753

Precision: 0.37093275488069416
Accuracy: 0.5059897318881916
AUC: 0.5155506015569711


--- SVM ---

In [17]:
from sklearn.svm import SVC
svc= SVC(kernel= 'rbf',probability= True, gamma= 1, random_state=15)
svc.fit(X_sm_tl_selected, y_sm_tl)
y_pred_svc= svc.predict(X_sm_tl_test_selected)
y_pred_prob_svc= svc.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_svc, labels= svc.classes_)
print(classification_report(y_test, y_pred_svc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_svc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_svc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_svc))

              precision    recall  f1-score   support

           0       0.65      0.24      0.36      1125
           1       0.36      0.77      0.49       628

    accuracy                           0.43      1753
   macro avg       0.51      0.51      0.42      1753
weighted avg       0.55      0.43      0.40      1753

Precision: 0.3623405851462866
Accuracy: 0.43240159726183686
AUC: 0.4964225053078556


--- Random Forest ---

In [18]:
from sklearn.ensemble import RandomForestClassifier
rf= RandomForestClassifier(random_state=15)
rf.fit(X_sm_tl_selected, y_sm_tl)
y_pred_rf= rf.predict(X_sm_tl_test_selected)
y_pred_prob_rf= rf.predict_proba(X_sm_tl_test_selected)[:,1]

In [19]:
cm= confusion_matrix(y_test, y_pred_rf, labels= rf.classes_)
print(classification_report(y_test, y_pred_rf, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_rf, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_rf))
print('AUC:', roc_auc_score(y_test, y_pred_prob_rf))

              precision    recall  f1-score   support

           0       0.65      0.13      0.22      1125
           1       0.36      0.87      0.51       628

    accuracy                           0.40      1753
   macro avg       0.50      0.50      0.36      1753
weighted avg       0.55      0.40      0.32      1753

Precision: 0.35952848722986247
Accuracy: 0.3970336565887051
AUC: 0.5055010615711253


In [20]:
param_grid= {
             'n_estimators': [100, 200, 300],
             'max_depth': [None, 10, 20, 30],
             'min_samples_split': [2, 5, 10],
             'min_samples_leaf': [1, 2, 4],
            }

gs_tuned_rf= GridSearchCV(estimator= rf, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_tuned_rf.fit(X_sm_tl, y_sm_tl)
print("Best Parameters:", gs_tuned_rf.best_params_)
print("Best Precision Score:", gs_tuned_rf.best_score_)

Fitting 5 folds for each of 108 candidates, totalling 540 fits
Best Parameters: {'max_depth': None, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 200}
Best Precision Score: 0.6525008606979478


In [21]:
tuned_rf= gs_tuned_rf.best_estimator_
tuned_rf.fit(X_sm_tl_selected, y_sm_tl)
y_pred_tuned_rf= tuned_rf.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_rf= tuned_rf.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_rf, labels= tuned_rf.classes_)
print(classification_report(y_test, y_pred_tuned_rf, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_rf, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_rf))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_rf))

              precision    recall  f1-score   support

           0       0.65      0.16      0.25      1125
           1       0.36      0.85      0.51       628

    accuracy                           0.41      1753
   macro avg       0.51      0.50      0.38      1753
weighted avg       0.55      0.41      0.34      1753

Precision: 0.3600809170600135
Accuracy: 0.4050199657729606
AUC: 0.504858457183298


--- AdaBoost ---

In [22]:
from sklearn.ensemble import AdaBoostClassifier
ada= AdaBoostClassifier(random_state=15)
ada.fit(X_sm_tl_selected, y_sm_tl)
y_pred_ada= ada.predict(X_sm_tl_test_selected)
y_pred_prob_ada= ada.predict_proba(X_sm_tl_test_selected)[:,1]

In [23]:
cm= confusion_matrix(y_test, y_pred_ada, labels= ada.classes_)
print(classification_report(y_test, y_pred_ada, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_ada, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_ada))
print('AUC:', roc_auc_score(y_test, y_pred_prob_ada))

              precision    recall  f1-score   support

           0       0.68      0.29      0.41      1125
           1       0.37      0.75      0.50       628

    accuracy                           0.46      1753
   macro avg       0.52      0.52      0.45      1753
weighted avg       0.57      0.46      0.44      1753

Precision: 0.37203791469194314
Accuracy: 0.4569309754706218
AUC: 0.5138365180467092


In [24]:
param_grid= {'n_estimators': [50, 100, 200]}

gs_ada= GridSearchCV(estimator= ada, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_ada.fit(X_sm_tl, y_sm_tl)
print("Best Parameters:", gs_ada.best_params_)
print("Best Precision Score:", gs_ada.best_score_)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'n_estimators': 200}
Best Precision Score: 0.6604585966622223


In [25]:
tuned_ada= gs_ada.best_estimator_
tuned_ada.fit(X_sm_tl_selected, y_sm_tl)
y_pred_tuned_ada= tuned_ada.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_ada= tuned_ada.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_ada, labels= tuned_ada.classes_)
print(classification_report(y_test, y_pred_tuned_ada, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_ada, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_ada))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_ada))

              precision    recall  f1-score   support

           0       0.66      0.17      0.27      1125
           1       0.36      0.85      0.51       628

    accuracy                           0.41      1753
   macro avg       0.51      0.51      0.39      1753
weighted avg       0.55      0.41      0.35      1753

Precision: 0.3619047619047619
Accuracy: 0.41015402167712495
AUC: 0.5112830856334042


--- GradientBoosting ---

In [26]:
from sklearn.ensemble import GradientBoostingClassifier
grb= GradientBoostingClassifier(random_state=15)
grb.fit(X_sm_tl_selected, y_sm_tl)
y_pred_grb= grb.predict(X_sm_tl_test_selected)
y_pred_prob_grb= grb.predict_proba(X_sm_tl_test_selected)[:,1]


cm= confusion_matrix(y_test, y_pred_grb, labels= grb.classes_)
print(classification_report(y_test, y_pred_grb, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_grb, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_grb))
print('AUC:', roc_auc_score(y_test, y_pred_prob_grb))

              precision    recall  f1-score   support

           0       0.67      0.17      0.27      1125
           1       0.36      0.85      0.51       628

    accuracy                           0.41      1753
   macro avg       0.52      0.51      0.39      1753
weighted avg       0.56      0.41      0.35      1753

Precision: 0.36363636363636365
Accuracy: 0.41243582430119796
AUC: 0.5102880396319887


In [27]:
param_grid= {'learning_rate': [0.01, 0.1, 0.2]}

gs_grb= GridSearchCV(estimator= grb, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_grb.fit(X_sm_tl, y_sm_tl)

print("Best Parameters:", gs_grb.best_params_)
print("Best Precision Score:", gs_grb.best_score_)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'learning_rate': 0.2}
Best Precision Score: 0.6728189788028935


In [28]:
tuned_grb= gs_grb.best_estimator_
tuned_grb.fit(X_sm_tl_selected, y_sm_tl)
y_pred_tuned_grb= tuned_grb.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_grb= tuned_grb.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_grb, labels= tuned_grb.classes_)
print(classification_report(y_test, y_pred_tuned_grb, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_grb, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_grb))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_grb))

              precision    recall  f1-score   support

           0       0.64      0.12      0.20      1125
           1       0.36      0.89      0.51       628

    accuracy                           0.39      1753
   macro avg       0.50      0.50      0.35      1753
weighted avg       0.54      0.39      0.31      1753

Precision: 0.3584784010315925
Accuracy: 0.39132915002852253
AUC: 0.5089476291578202


--- XGB ---

In [29]:
from xgboost import XGBClassifier
xgb= XGBClassifier(random_state=15)
xgb.fit(X_sm_tl_selected, y_sm_tl)
y_pred_xgb= xgb.predict(X_sm_tl_test_selected)
y_pred_prob_xgb= xgb.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_xgb, labels= xgb.classes_)
print(classification_report(y_test, y_pred_xgb, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_xgb, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_xgb))
print('AUC:', roc_auc_score(y_test, y_pred_prob_xgb))

              precision    recall  f1-score   support

           0       0.65      0.13      0.22      1125
           1       0.36      0.87      0.51       628

    accuracy                           0.40      1753
   macro avg       0.50      0.50      0.36      1753
weighted avg       0.55      0.40      0.32      1753

Precision: 0.35952848722986247
Accuracy: 0.3970336565887051
AUC: 0.49776291578202403


In [30]:
params_XGBoost= {'learning_rate': [0.01, 0.1, 1.0]}

gs_xgb= GridSearchCV(estimator= xgb, param_grid= params_XGBoost, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_xgb.fit(X_sm_tl, y_sm_tl)

print("Best Parameters:", gs_xgb.best_params_)
print("Best Precision Score:", gs_xgb.best_score_)

tuned_xgb= gs_xgb.best_estimator_
tuned_xgb.fit(X_sm_tl_selected, y_sm_tl)
y_pred_tuned_xgb= tuned_xgb.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_xgb= tuned_xgb.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_xgb, labels= tuned_xgb.classes_)
print(classification_report(y_test, y_pred_tuned_xgb, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_xgb, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_xgb))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_xgb))

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'learning_rate': 0.1}
Best Precision Score: 0.681213319906566
              precision    recall  f1-score   support

           0       0.65      0.13      0.22      1125
           1       0.36      0.87      0.51       628

    accuracy                           0.40      1753
   macro avg       0.50      0.50      0.36      1753
weighted avg       0.55      0.40      0.32      1753

Precision: 0.35952848722986247
Accuracy: 0.3970336565887051
AUC: 0.49834465675866957


--- LGBM ---

In [31]:
from lightgbm import LGBMClassifier
lgm= LGBMClassifier(random_state=15)
lgm.fit(X_sm_tl_selected, y_sm_tl)
y_pred_lgm= lgm.predict(X_sm_tl_test_selected)
y_pred_prob_lgm= lgm.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_lgm, labels= lgm.classes_)
print(classification_report(y_test, y_pred_lgm, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_lgm, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_lgm))
print('AUC:', roc_auc_score(y_test, y_pred_prob_lgm))

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 2511, number of negative: 2511
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003258 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 79
[LightGBM] [Info] Number of data points in the train set: 5022, number of used features: 4
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
              precision    recall  f1-score   support

           0       0.65      0.16      0.26      1125
           1       0.36      0.84      0.50       628

    accuracy                           0.41      1753
   macro avg       0.50      0.50      0.38      1753
weighted avg       0.55      0.41      0.35      1753

Precision: 0.35956580732700133
Accuracy: 0.4055904164289789
AUC: 0.49745718329794764


In [32]:
params_LGB= {'learning_rate': [0.001, 0.01, 0.1, 1.0],
             'num_leaves': [31, 127],
             'reg_alpha': [0.1, 0.5],
             'min_data_in_leaf': [30, 50, 100, 300, 400]}

gs_lgm= GridSearchCV(estimator= lgm, param_grid= params_LGB, cv=5, scoring='precision', n_jobs=-1, verbose=2)
gs_lgm.fit(X_sm_tl, y_sm_tl)

print("Best Parameters:", gs_lgm.best_params_)
print("Best Precision Score:", gs_lgm.best_score_)

tuned_lgm= gs_lgm.best_estimator_
tuned_lgm.fit(X_sm_tl_selected, y_sm_tl)
y_pred_tuned_lgm= tuned_lgm.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_lgm= tuned_lgm.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_lgm, labels= tuned_lgm.classes_)
print(classification_report(y_test, y_pred_tuned_lgm, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_lgm, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_lgm))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_lgm))

Fitting 5 folds for each of 80 candidates, totalling 400 fits
[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[LightGBM] [Info] Number of positive: 2511, number of negative: 2511
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000677 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2239
[LightGBM] [Info] Number of data points in the train set: 5022, number of used features: 22
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
Best Parameters: {'learning_rate': 0.1, 'min_data_in_leaf': 30, 'num_leaves': 31, 'reg_alpha': 0.1}
Best Precision Score: 0.6782596947381208
[LightGBM] [Warning] min_data_in_leaf is set=30, min_

--- CatBoost ---

In [33]:
from catboost import CatBoostClassifier
cat= CatBoostClassifier(random_state=15)
cat.fit(X_sm_tl_selected, y_sm_tl)
y_pred_cat= cat.predict(X_sm_tl_test_selected)
y_pred_prob_cat= cat.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_cat, labels= cat.classes_)
print(classification_report(y_test, y_pred_cat, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_cat, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_cat))
print('AUC:', roc_auc_score(y_test, y_pred_prob_cat))

Learning rate set to 0.020521
0:	learn: 0.6895921	total: 149ms	remaining: 2m 28s
1:	learn: 0.6872956	total: 153ms	remaining: 1m 16s
2:	learn: 0.6848351	total: 157ms	remaining: 52.1s
3:	learn: 0.6820074	total: 161ms	remaining: 40.1s
4:	learn: 0.6799675	total: 166ms	remaining: 32.9s
5:	learn: 0.6780706	total: 170ms	remaining: 28.2s
6:	learn: 0.6761643	total: 175ms	remaining: 24.9s
7:	learn: 0.6752095	total: 179ms	remaining: 22.2s
8:	learn: 0.6730025	total: 184ms	remaining: 20.3s
9:	learn: 0.6715362	total: 188ms	remaining: 18.6s
10:	learn: 0.6703802	total: 193ms	remaining: 17.4s
11:	learn: 0.6682246	total: 197ms	remaining: 16.2s
12:	learn: 0.6663962	total: 200ms	remaining: 15.2s
13:	learn: 0.6651142	total: 204ms	remaining: 14.4s
14:	learn: 0.6636342	total: 207ms	remaining: 13.6s
15:	learn: 0.6619591	total: 211ms	remaining: 13s
16:	learn: 0.6606315	total: 215ms	remaining: 12.4s
17:	learn: 0.6597663	total: 219ms	remaining: 12s
18:	learn: 0.6579965	total: 223ms	remaining: 11.5s
19:	learn: 0.

170:	learn: 0.5680494	total: 835ms	remaining: 4.05s
171:	learn: 0.5678809	total: 839ms	remaining: 4.04s
172:	learn: 0.5677763	total: 842ms	remaining: 4.03s
173:	learn: 0.5675681	total: 846ms	remaining: 4.02s
174:	learn: 0.5673814	total: 851ms	remaining: 4.01s
175:	learn: 0.5670758	total: 855ms	remaining: 4s
176:	learn: 0.5670179	total: 858ms	remaining: 3.99s
177:	learn: 0.5669674	total: 862ms	remaining: 3.98s
178:	learn: 0.5667739	total: 867ms	remaining: 3.98s
179:	learn: 0.5662196	total: 873ms	remaining: 3.98s
180:	learn: 0.5659911	total: 876ms	remaining: 3.96s
181:	learn: 0.5658029	total: 882ms	remaining: 3.96s
182:	learn: 0.5656421	total: 888ms	remaining: 3.96s
183:	learn: 0.5652889	total: 896ms	remaining: 3.97s
184:	learn: 0.5651149	total: 900ms	remaining: 3.96s
185:	learn: 0.5649457	total: 903ms	remaining: 3.95s
186:	learn: 0.5649171	total: 907ms	remaining: 3.94s
187:	learn: 0.5648868	total: 913ms	remaining: 3.94s
188:	learn: 0.5647637	total: 921ms	remaining: 3.95s
189:	learn: 0.5

336:	learn: 0.5506899	total: 1.53s	remaining: 3.01s
337:	learn: 0.5506767	total: 1.53s	remaining: 3s
338:	learn: 0.5506000	total: 1.54s	remaining: 3s
339:	learn: 0.5505165	total: 1.54s	remaining: 2.99s
340:	learn: 0.5505026	total: 1.54s	remaining: 2.98s
341:	learn: 0.5504080	total: 1.55s	remaining: 2.98s
342:	learn: 0.5503979	total: 1.55s	remaining: 2.97s
343:	learn: 0.5503140	total: 1.56s	remaining: 2.97s
344:	learn: 0.5502962	total: 1.56s	remaining: 2.96s
345:	learn: 0.5502773	total: 1.57s	remaining: 2.96s
346:	learn: 0.5502191	total: 1.57s	remaining: 2.96s
347:	learn: 0.5502052	total: 1.57s	remaining: 2.95s
348:	learn: 0.5501539	total: 1.58s	remaining: 2.94s
349:	learn: 0.5501331	total: 1.58s	remaining: 2.94s
350:	learn: 0.5501328	total: 1.58s	remaining: 2.93s
351:	learn: 0.5501257	total: 1.59s	remaining: 2.92s
352:	learn: 0.5500490	total: 1.59s	remaining: 2.92s
353:	learn: 0.5499541	total: 1.6s	remaining: 2.91s
354:	learn: 0.5499517	total: 1.6s	remaining: 2.91s
355:	learn: 0.549863

503:	learn: 0.5433173	total: 2.23s	remaining: 2.19s
504:	learn: 0.5432791	total: 2.23s	remaining: 2.19s
505:	learn: 0.5432130	total: 2.23s	remaining: 2.18s
506:	learn: 0.5431709	total: 2.24s	remaining: 2.18s
507:	learn: 0.5431615	total: 2.24s	remaining: 2.17s
508:	learn: 0.5431310	total: 2.25s	remaining: 2.17s
509:	learn: 0.5431274	total: 2.25s	remaining: 2.16s
510:	learn: 0.5430992	total: 2.25s	remaining: 2.16s
511:	learn: 0.5430298	total: 2.26s	remaining: 2.15s
512:	learn: 0.5430068	total: 2.26s	remaining: 2.15s
513:	learn: 0.5429404	total: 2.26s	remaining: 2.14s
514:	learn: 0.5428739	total: 2.27s	remaining: 2.14s
515:	learn: 0.5428473	total: 2.27s	remaining: 2.13s
516:	learn: 0.5427512	total: 2.27s	remaining: 2.13s
517:	learn: 0.5427110	total: 2.28s	remaining: 2.12s
518:	learn: 0.5426924	total: 2.28s	remaining: 2.12s
519:	learn: 0.5426447	total: 2.29s	remaining: 2.11s
520:	learn: 0.5425615	total: 2.29s	remaining: 2.1s
521:	learn: 0.5424950	total: 2.29s	remaining: 2.1s
522:	learn: 0.

677:	learn: 0.5384535	total: 2.92s	remaining: 1.39s
678:	learn: 0.5384421	total: 2.92s	remaining: 1.38s
679:	learn: 0.5384241	total: 2.93s	remaining: 1.38s
680:	learn: 0.5384199	total: 2.93s	remaining: 1.37s
681:	learn: 0.5384148	total: 2.94s	remaining: 1.37s
682:	learn: 0.5384007	total: 2.94s	remaining: 1.36s
683:	learn: 0.5383903	total: 2.94s	remaining: 1.36s
684:	learn: 0.5383606	total: 2.95s	remaining: 1.35s
685:	learn: 0.5383344	total: 2.95s	remaining: 1.35s
686:	learn: 0.5383330	total: 2.96s	remaining: 1.35s
687:	learn: 0.5383154	total: 2.96s	remaining: 1.34s
688:	learn: 0.5382865	total: 2.96s	remaining: 1.34s
689:	learn: 0.5382707	total: 2.97s	remaining: 1.33s
690:	learn: 0.5382638	total: 2.97s	remaining: 1.33s
691:	learn: 0.5382510	total: 2.98s	remaining: 1.32s
692:	learn: 0.5382328	total: 2.98s	remaining: 1.32s
693:	learn: 0.5382196	total: 2.98s	remaining: 1.32s
694:	learn: 0.5382136	total: 2.99s	remaining: 1.31s
695:	learn: 0.5381982	total: 2.99s	remaining: 1.31s
696:	learn: 

852:	learn: 0.5370254	total: 3.62s	remaining: 623ms
853:	learn: 0.5370097	total: 3.62s	remaining: 619ms
854:	learn: 0.5370060	total: 3.62s	remaining: 615ms
855:	learn: 0.5369996	total: 3.63s	remaining: 610ms
856:	learn: 0.5369900	total: 3.63s	remaining: 606ms
857:	learn: 0.5369879	total: 3.64s	remaining: 602ms
858:	learn: 0.5369816	total: 3.64s	remaining: 598ms
859:	learn: 0.5369805	total: 3.65s	remaining: 593ms
860:	learn: 0.5369740	total: 3.65s	remaining: 589ms
861:	learn: 0.5369650	total: 3.65s	remaining: 585ms
862:	learn: 0.5369646	total: 3.66s	remaining: 581ms
863:	learn: 0.5369632	total: 3.66s	remaining: 576ms
864:	learn: 0.5369620	total: 3.67s	remaining: 572ms
865:	learn: 0.5369551	total: 3.67s	remaining: 568ms
866:	learn: 0.5369533	total: 3.67s	remaining: 564ms
867:	learn: 0.5369519	total: 3.68s	remaining: 559ms
868:	learn: 0.5369396	total: 3.68s	remaining: 555ms
869:	learn: 0.5369373	total: 3.69s	remaining: 551ms
870:	learn: 0.5369286	total: 3.69s	remaining: 546ms
871:	learn: 

In [34]:
params_CatBoost= {
                  'depth': [3,5,10],
                  'learning_rate' : [0.01,0.1,1],
                  'iterations' : [5,10,50,100]
                 }

gs_cat= GridSearchCV(estimator= cat, param_grid= params_CatBoost, cv=5, scoring='precision', n_jobs=-1, verbose=2)
gs_cat.fit(X_sm_tl, y_sm_tl)

print("Best Parameters:", gs_cat.best_params_)
print("Best Precision Score:", gs_cat.best_score_)

tuned_cat= gs_cat.best_estimator_
tuned_cat.fit(X_sm_tl_selected, y_sm_tl)
y_pred_tuned_cat= tuned_cat.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_cat= tuned_cat.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_cat, labels= tuned_cat.classes_)
print(classification_report(y_test, y_pred_tuned_cat, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_cat, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_cat))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_cat))

Fitting 5 folds for each of 36 candidates, totalling 180 fits
0:	learn: 0.6142089	total: 4.8ms	remaining: 475ms
1:	learn: 0.5964857	total: 9.31ms	remaining: 456ms
2:	learn: 0.5548629	total: 15.3ms	remaining: 493ms
3:	learn: 0.5410952	total: 19.9ms	remaining: 477ms
4:	learn: 0.5318377	total: 24.9ms	remaining: 472ms
5:	learn: 0.5280988	total: 30.5ms	remaining: 478ms
6:	learn: 0.5214196	total: 35.4ms	remaining: 470ms
7:	learn: 0.5169404	total: 40.8ms	remaining: 469ms
8:	learn: 0.5115407	total: 52.8ms	remaining: 534ms
9:	learn: 0.5066817	total: 62.8ms	remaining: 565ms
10:	learn: 0.5001106	total: 72.8ms	remaining: 589ms
11:	learn: 0.4922419	total: 84.9ms	remaining: 623ms
12:	learn: 0.4849907	total: 96.3ms	remaining: 645ms
13:	learn: 0.4771158	total: 105ms	remaining: 647ms
14:	learn: 0.4704775	total: 115ms	remaining: 654ms
15:	learn: 0.4641021	total: 124ms	remaining: 652ms
16:	learn: 0.4577852	total: 134ms	remaining: 654ms
17:	learn: 0.4469836	total: 145ms	remaining: 659ms
18:	learn: 0.44066

62:	learn: 0.5370700	total: 225ms	remaining: 132ms
63:	learn: 0.5370572	total: 228ms	remaining: 128ms
64:	learn: 0.5370392	total: 232ms	remaining: 125ms
65:	learn: 0.5369376	total: 236ms	remaining: 121ms
66:	learn: 0.5369279	total: 240ms	remaining: 118ms
67:	learn: 0.5369151	total: 244ms	remaining: 115ms
68:	learn: 0.5368433	total: 247ms	remaining: 111ms
69:	learn: 0.5368268	total: 251ms	remaining: 108ms
70:	learn: 0.5368170	total: 255ms	remaining: 104ms
71:	learn: 0.5367848	total: 257ms	remaining: 100ms
72:	learn: 0.5366823	total: 261ms	remaining: 96.5ms
73:	learn: 0.5366767	total: 264ms	remaining: 92.8ms
74:	learn: 0.5366341	total: 268ms	remaining: 89.3ms
75:	learn: 0.5366246	total: 271ms	remaining: 85.6ms
76:	learn: 0.5366010	total: 274ms	remaining: 81.8ms
77:	learn: 0.5365707	total: 278ms	remaining: 78.4ms
78:	learn: 0.5365381	total: 281ms	remaining: 74.8ms
79:	learn: 0.5365214	total: 285ms	remaining: 71.3ms
80:	learn: 0.5364868	total: 291ms	remaining: 68.2ms
81:	learn: 0.5364843	t

--- Visualisation ---

In [35]:
precision_scores= {
                    'Logistic Regression Precision:': precision_score(y_test, y_pred_lr, zero_division= 0),
                    'Decision Tree Precision:': precision_score(y_test, y_pred_dt, zero_division= 0),
                    'Tuned Decision Tree Precision:': precision_score(y_test, y_pred_tuned_dt, zero_division= 0),
                    'KNeighborsClassifier Precision:': precision_score(y_test, y_pred_knn, zero_division= 0),
                    'Tuned KNeighborsClassifier Precision:': precision_score(y_test, y_pred_tuned_knn, zero_division= 0),
                    'GaussianNB Precision:': precision_score(y_test, y_pred_nb, zero_division= 0),
                    'SVM Precision:': precision_score(y_test, y_pred_svc, zero_division= 0),
                    'Random Forest Precision:': precision_score(y_test, y_pred_rf, zero_division= 0),
                    'Tuned Random Forest Precision:': precision_score(y_test, y_pred_tuned_rf, zero_division= 0),
                    'AdaBoost Precision:': precision_score(y_test, y_pred_ada, zero_division= 0),
                    'Tuned AdaBoost Precision:': precision_score(y_test, y_pred_tuned_ada, zero_division= 0),
                    'GradientBoosting Precision:': precision_score(y_test, y_pred_grb, zero_division= 0),
                    'Tuned GradientBoosting Precision:': precision_score(y_test, y_pred_tuned_grb, zero_division= 0),
                    'XGB Precision:': precision_score(y_test, y_pred_xgb, zero_division= 0),
                    'Tuned XGB Precision:': precision_score(y_test, y_pred_tuned_xgb, zero_division= 0),
                    'LGBM Precision:': precision_score(y_test, y_pred_lgm, zero_division= 0),
                    'Tuned LGBM Precision:': precision_score(y_test, y_pred_tuned_lgm, zero_division= 0),
                    'CatBoost Precision:': precision_score(y_test, y_pred_cat, zero_division= 0),
                    'Tuned CatBoost Precision:': precision_score(y_test, y_pred_tuned_cat, zero_division= 0)
                  }

be_precision= pd.DataFrame(list(precision_scores.items()), columns= ['Model', 'Precision Score'])
be_precision= be_precision.sort_values(by= 'Precision Score', ascending=False)
print(be_precision)

                                    Model  Precision Score
4   Tuned KNeighborsClassifier Precision:         0.391549
3         KNeighborsClassifier Precision:         0.383721
9                     AdaBoost Precision:         0.372038
5                   GaussianNB Precision:         0.370933
0          Logistic Regression Precision:         0.370933
11            GradientBoosting Precision:         0.363636
6                          SVM Precision:         0.362341
10              Tuned AdaBoost Precision:         0.361905
2          Tuned Decision Tree Precision:         0.360667
8          Tuned Random Forest Precision:         0.360081
15                        LGBM Precision:         0.359566
16                  Tuned LGBM Precision:         0.359566
7                Random Forest Precision:         0.359528
1                Decision Tree Precision:         0.359528
13                         XGB Precision:         0.359528
14                   Tuned XGB Precision:         0.3595

In [36]:
accuracy_scores= {
                    'Logistic Regression Accuracy:': accuracy_score(y_test, y_pred_lr),
                    'Decision Tree Accuracy:': accuracy_score(y_test, y_pred_dt),
                    'Tuned Decision Tree Accuracy:': accuracy_score(y_test, y_pred_tuned_dt),
                    'KNeighborsClassifier Accuracy:': accuracy_score(y_test, y_pred_knn),
                    'Tuned KNeighborsClassifier Accuracy:': accuracy_score(y_test, y_pred_tuned_knn),
                    'GaussianNB Accuracy:': accuracy_score(y_test, y_pred_nb),
                    'SVM Accuracy:': accuracy_score(y_test, y_pred_svc),
                    'Random Forest Accuracy:': accuracy_score(y_test, y_pred_rf),
                    'Tuned Random Forest Accuracy:': accuracy_score(y_test, y_pred_tuned_rf),
                    'AdaBoost Accuracy:': accuracy_score(y_test, y_pred_ada),
                    'Tuned AdaBoost Accuracy:': accuracy_score(y_test, y_pred_tuned_ada),
                    'GradientBoosting Accuracy:': accuracy_score(y_test, y_pred_grb),
                    'Tuned GradientBoosting Accuracy:': accuracy_score(y_test, y_pred_tuned_grb),
                    'XGB Accuracy:': accuracy_score(y_test, y_pred_xgb),
                    'Tuned XGB Accuracy:': accuracy_score(y_test, y_pred_tuned_xgb),
                    'LGBM Accuracy:': accuracy_score(y_test, y_pred_lgm),
                    'Tuned LGBM Accuracy:': accuracy_score(y_test, y_pred_tuned_lgm),
                    'CatBoost Accuracy:': accuracy_score(y_test, y_pred_cat),
                    'Tuned CatBoost Accuracy:': accuracy_score(y_test, y_pred_tuned_cat)
                  }

be_accuracy= pd.DataFrame(list(accuracy_scores.items()), columns= ['Model', 'Accuracy Score'])
be_accuracy= be_accuracy.sort_values(by= 'Accuracy Score', ascending=False)
print(be_accuracy)

                                   Model  Accuracy Score
3         KNeighborsClassifier Accuracy:        0.607530
4   Tuned KNeighborsClassifier Accuracy:        0.597832
0          Logistic Regression Accuracy:        0.505990
5                   GaussianNB Accuracy:        0.505990
9                     AdaBoost Accuracy:        0.456931
6                          SVM Accuracy:        0.432402
11            GradientBoosting Accuracy:        0.412436
10              Tuned AdaBoost Accuracy:        0.410154
15                        LGBM Accuracy:        0.405590
16                  Tuned LGBM Accuracy:        0.405590
8          Tuned Random Forest Accuracy:        0.405020
2          Tuned Decision Tree Accuracy:        0.403309
18              Tuned CatBoost Accuracy:        0.401027
7                Random Forest Accuracy:        0.397034
1                Decision Tree Accuracy:        0.397034
13                         XGB Accuracy:        0.397034
14                   Tuned XGB 

In [37]:
auc_scores= {
                    'Logistic Regression AUC:': roc_auc_score(y_test, y_pred_prob_lr),
                    'Decision Tree AUC:': roc_auc_score(y_test, y_pred_prob_dt),
                    'Tuned Decision Tree AUC:': roc_auc_score(y_test, y_pred_prob_tuned_dt),
                    'KNeighborsClassifier AUC:': roc_auc_score(y_test, y_pred_prob_knn),
                    'Tuned KNeighborsClassifier AUC:': roc_auc_score(y_test, y_pred_prob_tuned_knn),
                    'GaussianNB AUC:': roc_auc_score(y_test, y_pred_prob_nb),
                    'SVM AUC:': roc_auc_score(y_test, y_pred_prob_svc),
                    'Random Forest AUC:': roc_auc_score(y_test, y_pred_prob_rf),
                    'Tuned Random Forest AUC:': roc_auc_score(y_test, y_pred_prob_tuned_rf),
                    'AdaBoost AUC:': roc_auc_score(y_test, y_pred_prob_ada),
                    'Tuned AdaBoost AUC:': roc_auc_score(y_test, y_pred_prob_tuned_ada),
                    'GradientBoosting AUC:': roc_auc_score(y_test, y_pred_prob_grb),
                    'Tuned GradientBoosting AUC:': roc_auc_score(y_test, y_pred_prob_tuned_grb),
                    'XGB AUC:': roc_auc_score(y_test, y_pred_prob_xgb),
                    'Tuned XGB AUC:': roc_auc_score(y_test, y_pred_prob_tuned_xgb),
                    'LGBM AUC:': roc_auc_score(y_test, y_pred_prob_lgm),
                    'Tuned LGBM AUC:': roc_auc_score(y_test, y_pred_prob_tuned_lgm),
                    'CatBoost AUC:': roc_auc_score(y_test, y_pred_prob_cat),
                    'Tuned CatBoost AUC:': roc_auc_score(y_test, y_pred_prob_tuned_cat)
                  }

be_auc= pd.DataFrame(list(auc_scores.items()), columns= ['Model', 'AUC Score'])
be_auc= be_auc.sort_values(by= 'AUC Score', ascending=False)
print(be_auc)

                              Model  AUC Score
0          Logistic Regression AUC:   0.516475
5                   GaussianNB AUC:   0.515551
9                     AdaBoost AUC:   0.513837
10              Tuned AdaBoost AUC:   0.511283
11            GradientBoosting AUC:   0.510288
2          Tuned Decision Tree AUC:   0.509137
12      Tuned GradientBoosting AUC:   0.508948
3         KNeighborsClassifier AUC:   0.508155
7                Random Forest AUC:   0.505501
8          Tuned Random Forest AUC:   0.504858
1                Decision Tree AUC:   0.504233
17                    CatBoost AUC:   0.499290
14                   Tuned XGB AUC:   0.498345
16                  Tuned LGBM AUC:   0.498206
18              Tuned CatBoost AUC:   0.498175
13                         XGB AUC:   0.497763
15                        LGBM AUC:   0.497457
6                          SVM AUC:   0.496423
4   Tuned KNeighborsClassifier AUC:   0.493832
